In [2]:
from numpy import array
import pickle
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten

from sklearn.preprocessing import OneHotEncoder
import gensim 
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors



Using TensorFlow backend.


In [3]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


variable

In [4]:
# path
checkpoint_path = '/content/drive/My Drive/Colab Notebooks/Language Model/Language Models/save checkpoints/{0}'
data_path = '/content/drive/My Drive/Colab Notebooks/Language Model/Language Models/data/{0}'
model_path = '/content/drive/My Drive/Colab Notebooks/Language Model/Language Models/embedding model/{0}'
pre_train_path = '/content/drive/My Drive/Colab Notebooks/Language Model/Language Models/pre-train model/Glove/{0}'

In [5]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r', encoding='utf8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [6]:
# load
in_filename = data_path.format('01. The Last Wish, The - Andrzej Sapkowski_clean.txt')
doc = load_doc(in_filename)
lines = doc.split('\n')

In [7]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [8]:
# print(doc)

In [9]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

## embedding style

#### one hot encoding

In [10]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [11]:
print(X.shape)

(113888, 50)


In [12]:
print(y.shape)

(113888, 3921)


Convert pre-train model elmo

In [15]:
# model = Word2Vec.load(pre_train_path.format('baomoi_model.bin'))

# model = KeyedVectors.load_word2vec_format(pre_train_path.format('saved_model.pb'), binary=True, unicode_errors='ignore')

# model = open(pre_train_path.format('Glove_wiki.vi.model.bin'), encoding='utf-8')
# model.wv.save_word2vec_format(pre_train_path.format('saved_model.txt'), binary=False)

### Word Vector

#### Glove encoding

In [16]:
# pad documents to a max length of 4 words
max_length = 20
padded_docs = pad_sequences(sequences, maxlen=max_length, padding='post')
print(padded_docs)
# load the whole embedding into memory
embeddings_index = dict()
f = open(pre_train_path.format('Glove_wiki.vi.model.txt'), encoding='utf-8')
print(f)
for line in f:
	values = line.split()
	# print(values)
	word = values[0]
	coefs = asarray(values[1:], 
	                # dtype='float32'
									)
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 400))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

[[ 834 2920 2919 ...  834 2914  995]
 [2920 2919  834 ... 2914  995  834]
 [2919  834 1294 ...  995  834 2492]
 ...
 [ 142  566  336 ...  888    3   64]
 [ 566  336 1427 ...    3   64   30]
 [ 336 1427 1034 ...   64   30   94]]
<_io.TextIOWrapper name='/content/drive/My Drive/Colab Notebooks/Language Model/Language Models/pre-train model/Glove/Glove_wiki.vi.model.txt' mode='r' encoding='utf-8'>
Loaded 231487 word vectors.


In [17]:
Glove_model = Embedding(vocab_size, 400, weights=[embedding_matrix], input_length=50, trainable=False)
# evaluate the model
# loss, accuracy = Glove_model.evaluate(padded_docs, labels, verbose=0)
# loss, accuracy = Glove_model.evaluate(X, y, verbose=0)
# print('Accuracy: %f' % (accuracy*100))

In [18]:
# print(Glove_model.shape)

Build model

In [19]:
# checkpoint
# cp_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_path, 
#     verbose=1, 
#     save_weights_only=True,
#     save_freq='epoch')

In [20]:
# define model
model = Sequential()
# Glove embedding
model.add(Glove_model)

model.save_weights(checkpoint_path.format('language_model_glove_weight_ckpt.ckpt'))

model.add(LSTM(300, return_sequences=True))
model.add(LSTM(300))
model.add(Dense(400, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [21]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 400)           1568400   
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 300)           841200    
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_1 (Dense)              (None, 400)               120400    
_________________________________________________________________
dense_2 (Dense)              (None, 3921)              1572321   
Total params: 4,823,521
Trainable params: 3,255,121
Non-trainable params: 1,568,400
_________________________________________________________________
None


In [22]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
# fit model
model.fit(
    X, 
    y, 
    batch_size=128, 
    epochs=17,
    # callbacks=[cp_callback]
    )

# fit model glove
# model.fit(padded_docs, max_length, batch_size=128, epochs=100)

Epoch 1/20
113888/113888 [==============================] - 1279s 11ms/step - loss: 6.3909 - accuracy: 0.0468
Epoch 2/20
113888/113888 [==============================] - 1274s 11ms/step - loss: 5.6073 - accuracy: 0.1032
Epoch 3/20
113888/113888 [==============================] - 1281s 11ms/step - loss: 5.1119 - accuracy: 0.1446
Epoch 4/20
113888/113888 [==============================] - 1286s 11ms/step - loss: 4.7447 - accuracy: 0.1745
Epoch 5/20
113888/113888 [==============================] - 1278s 11ms/step - loss: 4.4364 - accuracy: 0.1979
Epoch 6/20
113888/113888 [==============================] - 1286s 11ms/step - loss: 4.1551 - accuracy: 0.2199
Epoch 7/20
113888/113888 [==============================] - 1288s 11ms/step - loss: 3.8864 - accuracy: 0.2443
Epoch 8/20
113888/113888 [==============================] - 1288s 11ms/step - loss: 3.6316 - accuracy: 0.2699
Epoch 9/20
113888/113888 [==============================] - 1291s 11ms/step - loss: 3.3820 - accuracy: 0.2986
Epoch 10/2

In [24]:
# save the model to file
# model.save(checkpoint_path.format('Language_model_glove_eng_final.h5'))
# save model vietnamese
model.save(checkpoint_path.format('Language_model_glove_vn_final.h5'))

In [25]:
# save the tokenizer
# dump(tokenizer, open(checkpoint_path.format('tokenizer_glove_eng.pkl'), 'wb'))

dump(tokenizer, open(checkpoint_path.format('tokenizer_glove_vn.pkl'), 'wb'))